# LLM-as-a-Judge Evaluation with Flex-Evals

This notebook demonstrates how to use Large Language Models (LLMs) to evaluate AI system outputs using the Flexible Evaluation Protocol (FEP). LLM-as-a-judge is particularly powerful for evaluating subjective qualities like helpfulness, accuracy, tone, and coherence that are difficult to capture with traditional programmatic checks.

## Why LLM-as-a-Judge?

Traditional evaluation metrics work well for objective criteria (exact matches, thresholds, etc.), but many real-world AI applications need evaluation of subjective qualities:

- **Helpfulness**: Is the response useful to the user?
- **Accuracy**: Does the response contain correct information?
- **Clarity**: Is the explanation easy to understand?
- **Completeness**: Does the response fully address the question?
- **Tone**: Is the response appropriate for the context?

LLM-as-a-judge allows us to evaluate these nuanced criteria at scale while maintaining consistency and auditability.

In [1]:
# Install required packages if needed: pip install flex-evals sik-llms pydantic
#
# IMPORTANT: This notebook requires valid LLM API credentials to run the judge function.
# If you're seeing N/A values or errors, please ensure:
# 1. You have API credentials configured (e.g., OPENAI_API_KEY environment variable)
# 2. The sik-llms package is properly configured with your LLM provider
# 3. Your API key has sufficient quota/credits
#
# The examples will show the structure and flow even without API access, but actual
# LLM judge evaluations require a working LLM connection.

In [2]:
from typing import Any
from pydantic import BaseModel, Field
from sik_llms import create_client, system_message, user_message

from flex_evals import evaluate
from flex_evals.schemas import TestCase, Output
from flex_evals import LLMJudgeCheck

import nest_asyncio
nest_asyncio.apply()  # for running async function in a Jupyter notebook

## Step 1: Define Evaluation Criteria

We use Pydantic models to define the structure of our evaluation results. This ensures type safety and provides clear documentation of what the LLM judge should evaluate.

In [3]:
class BinaryEvaluation(BaseModel):
    """Simple yes/no evaluation with reasoning."""

    answers_question: bool = Field(description="Whether the response answers the user's question")
    reasoning: str = Field(description="Brief explanation of the evaluation")


class DetailedQualityAssessment(BaseModel):
    """Comprehensive quality evaluation with multiple criteria."""

    overall_score: int = Field(ge=1, le=5, description="Overall quality score from 1 (poor) to 5 (excellent)")  # noqa: E501
    helpfulness: int = Field(ge=1, le=5, description="How helpful is the response to the user?")
    accuracy: int = Field(ge=1, le=5, description="How accurate is the information provided?")
    clarity: int = Field(ge=1, le=5, description="How clear and easy to understand is the response?")  # noqa: E501
    completeness: int = Field(ge=1, le=5, description="How completely does the response address the question?")  # noqa: E501
    strengths: list[str] = Field(description="Key strengths of the response")
    weaknesses: list[str] = Field(description="Areas for improvement")
    recommendation: str = Field(description="Overall recommendation for this response quality")

## Step 2: Create LLM Judge Function

We create a function that uses the sik_llms framework to call an LLM for evaluation. This function will be passed to our LLM judge check.

In [4]:
async def create_llm_judge_function(model_name: str = 'gpt-4o-mini') -> callable:
    """Create an LLM judge function using sik_llms framework."""

    async def llm_judge(prompt: str, response_format: type[BaseModel]) -> tuple[BaseModel, dict[str, Any]]:  # noqa: E501
        """
        LLM judge function that evaluates based on the given prompt.

        Returns:
            Tuple of (parsed_response, metadata_dict)
        """
        # Create client with the specified response format
        client = create_client(
            model_name=model_name,
            response_format=response_format,
        )

        # Create messages for the evaluation
        messages = [
            system_message(
                "You are an expert evaluator tasked with assessing AI responses. "
                "Provide objective, fair, and constructive evaluations based on the given criteria. "  # noqa: E501
                "Be specific in your reasoning and provide actionable feedback.",
            ),
            user_message(prompt),
        ]

        # Get evaluation from LLM
        response = await client.run_async(messages=messages)

        # Extract metadata from the response
        metadata = {
            "cost_usd": response.input_cost + response.output_cost,
            "tokens_used": response.input_tokens + response.output_tokens,
            "input_tokens": response.input_tokens,
            "output_tokens": response.output_tokens,
            "response_time_ms": int(response.duration_seconds * 1000),
            "model_version": model_name,
        }

        return response.parsed, metadata

    return llm_judge


# Create our judge function
llm_judge_function = await create_llm_judge_function()

## Step 3: Simple Example - Does the AI Answer the Question?

Let's start with a simple binary evaluation: does the AI response actually answer the user's question?

In [5]:
# Define the llm-as-a-judge check we want to perform on the AI responses
# Since we are using JSONPath to access test case input and expected values,
# we can reuse the same check for multiple test cases
llm_judge_check = LLMJudgeCheck(
    # Note that we use JSONPath to access test case input and expected values
    prompt="""
        Evaluate whether the AI response properly answers the user's question.

        **User Question:** {{$.test_case.input}}
        **AI Response:** {{$.output.value}}
        **Expected Answer:** {{$.test_case.expected}}

        Consider:
        - Does the response directly address the question?
        - Is the core information correct?
        - Is the response complete enough to be useful?
        """,
    response_format=BinaryEvaluation,
    llm_function=llm_judge_function,
)

# Create test cases for question-answering evaluation
simple_test_cases = [
    TestCase(
        id="qa_001",
        input="What is the capital of France?",
        expected="Paris",  # We know the correct answer
        metadata={"category": "geography", "difficulty": "easy"},
    ),
    TestCase(
        id="qa_002",
        input="How do I bake a chocolate cake?",
        expected="Step-by-step baking instructions",
        metadata={"category": "cooking", "difficulty": "medium"},
    ),
    TestCase(
        id="qa_003",
        input="What's the weather like?",
        expected="Request for location or explanation of inability to provide weather",
        metadata={"category": "general", "difficulty": "easy"},
    ),
]

# Simulated AI responses (in practice, these would come from your AI system)
simple_outputs = [
    Output(
        value="The capital of France is Paris.",
        metadata={"model": "gpt-4", "confidence": 0.99},
    ),
    Output(
        value="To bake a chocolate cake: 1) Preheat oven to 350°F, 2) Mix dry ingredients, 3) Add wet ingredients, 4) Bake for 30-35 minutes. You'll need flour, sugar, cocoa powder, eggs, butter, and baking powder.",  # noqa: E501
        metadata={"model": "gpt-4", "confidence": 0.95},
    ),
    Output(
        value="I can't provide current weather information as I don't have access to real-time data. Could you please specify your location and I can suggest ways to check the current weather?",  # noqa: E501
        metadata={"model": "gpt-4", "confidence": 0.90},
    ),
]

In [6]:
# Run the simple evaluation
print("🔍 Running Simple Question-Answering Evaluation...\n")

simple_results = evaluate(
    test_cases=simple_test_cases,
    outputs=simple_outputs,
    checks=[llm_judge_check],
)

# Display results
print("📊 Evaluation Results:")
print(f"   Total test cases: {simple_results.summary.total_test_cases}")
print(f"   Completed: {simple_results.summary.completed_test_cases}")
print(f"   Errors: {simple_results.summary.error_test_cases}")
print(f"   Overall status: {simple_results.status}\n")

# Show detailed results for each test case
for i, result in enumerate(simple_results.results):
    test_case = simple_test_cases[i]
    check_result = result.check_results[0]

    # The response fields are now directly in results
    results = check_result.results
    metadata = results.get('judge_metadata', {})

    print(f"🧪 Test Case: {test_case.id} ({test_case.metadata['category']})")
    print(f"   Question: {test_case.input}")
    print(f"   AI Response: {simple_outputs[i].value[:100]}{'...' if len(simple_outputs[i].value) > 100 else ''}")  # noqa: E501
    print(f"   ✅ Answers Question: {results.get('answers_question', 'N/A')}")
    print(f"   💭 Reasoning: {results.get('reasoning', 'N/A')}")

    # Display metadata if available
    if metadata:
        print("   📊 Judge Metadata:")
        print(f"      - Cost: ${metadata.get('cost_usd', 0):.4f}")
        print(f"      - Tokens: {metadata.get('tokens_used', 0)}")
        print(f"      - Response Time: {metadata.get('response_time_ms', 0)}ms")
        print(f"      - Model: {metadata.get('model_version', 'unknown')}")
    print()

🔍 Running Simple Question-Answering Evaluation...

📊 Evaluation Results:
   Total test cases: 3
   Completed: 3
   Errors: 0
   Overall status: completed

🧪 Test Case: qa_001 (geography)
   Question: What is the capital of France?
   AI Response: The capital of France is Paris.
   ✅ Answers Question: True
   💭 Reasoning: The AI response accurately answers the user's question by providing the correct capital of France, which is Paris. The information is direct and complete.
   📊 Judge Metadata:
      - Cost: $0.0001
      - Tokens: 234
      - Response Time: 1195ms
      - Model: gpt-4o-mini

🧪 Test Case: qa_002 (cooking)
   Question: How do I bake a chocolate cake?
   AI Response: To bake a chocolate cake: 1) Preheat oven to 350°F, 2) Mix dry ingredients, 3) Add wet ingredients, ...
   ✅ Answers Question: True
   💭 Reasoning: The AI response directly answers the user's question by providing a general step-by-step guide to baking a chocolate cake. The core information about ingredients 

## Step 4: Complex Example - Multi-Criteria Quality Assessment

Now let's do a more sophisticated evaluation that assesses multiple quality dimensions and provides detailed feedback.

In [7]:
# Create test cases for detailed quality assessment
# Unlike the previous example, each test case has it's own llm-as-a-judge check
complex_test_cases = [
    TestCase(
        id="quality_001",
        input={
            "question": "Can you explain how machine learning works?",
            "user_context": "I'm a beginner with no technical background",
            "desired_length": "medium explanation",
        },
        expected={
            "key_concepts": ["training", "data", "patterns", "predictions"],
            "complexity_level": "beginner-friendly",
            "includes_examples": True,
        },
        metadata={"domain": "AI/ML", "audience": "beginner"},
        checks=[
            LLMJudgeCheck(
                prompt="""
                Provide a comprehensive quality assessment of this AI response.

                **Context:**
                - User Question: {{$.test_case.input.question}}
                - User Background: {{$.test_case.input.user_context}}
                - Desired Length: {{$.test_case.input.desired_length}}
                - Target Audience: {{$.test_case.metadata.audience}}

                **AI Response to Evaluate:**
                {{$.output.value}}

                **Expected Elements:**
                - Key concepts to cover: {{$.test_case.expected.key_concepts}}
                - Complexity level: {{$.test_case.expected.complexity_level}}
                - Should include examples: {{$.test_case.expected.includes_examples}}

                **Evaluation Criteria:**

                1. **Helpfulness (1-5)**: How useful is this response to someone with the user's background?
                2. **Accuracy (1-5)**: Is the technical information correct and up-to-date?
                3. **Clarity (1-5)**: Is the explanation clear and easy to understand for the target audience?
                4. **Completeness (1-5)**: Does it adequately cover the topic without being overwhelming?

                Provide specific feedback on strengths and areas for improvement.
                """,  # noqa: E501
                response_format=DetailedQualityAssessment,
                llm_function=llm_judge_function,
            ),
        ],
    ),
    TestCase(
        id="quality_002",
        input={
            "question": "How do I fix my code that keeps crashing?",
            "user_context": "Intermediate programmer, Python development",
            "desired_length": "concise but complete",
        },
        expected={
            "key_concepts": ["debugging", "error messages", "troubleshooting steps"],
            "complexity_level": "intermediate",
            "includes_examples": True,
        },
        metadata={"domain": "programming", "audience": "intermediate"},
        checks=[
            LLMJudgeCheck(
                prompt="""
                Evaluate this programming help response for technical accuracy and usefulness.

                **Context:**
                - User Question: {{$.test_case.input.question}}
                - User Background: {{$.test_case.input.user_context}}
                - Desired Style: {{$.test_case.input.desired_length}}
                - Domain: {{$.test_case.metadata.domain}}

                **AI Response to Evaluate:**
                {{$.output.value}}

                **Expected Elements:**
                - Should cover: {{$.test_case.expected.key_concepts}}
                - Complexity: {{$.test_case.expected.complexity_level}}
                - Examples needed: {{$.test_case.expected.includes_examples}}

                Focus on whether the response provides actionable debugging advice appropriate for an intermediate programmer.
                """,  # noqa: E501
                response_format=DetailedQualityAssessment,
                llm_function=llm_judge_function,
            ),
        ],
    ),
]

# Simulated AI responses for complex evaluation
complex_outputs = [
    Output(
        value="""Machine learning is like teaching a computer to recognize patterns, similar to how you might learn to recognize your friends' faces.

Here's how it works in simple terms:

1. **Training with Data**: First, we show the computer lots of examples. For instance, if we want it to recognize cats, we show it thousands of cat photos labeled "cat."

2. **Finding Patterns**: The computer looks for common features in all those cat photos - things like pointy ears, whiskers, or fur patterns.

3. **Making Predictions**: When you show it a new photo, it uses those patterns to guess whether it's a cat or not.

Think of it like learning to cook - the more recipes you follow (training data), the better you get at creating new dishes (making predictions). The computer gets "smarter" by seeing more examples, just like you get better at cooking with practice!

This same process helps computers do amazing things like recommend movies on Netflix, translate languages, or even help doctors diagnose diseases.""",  # noqa: E501
        metadata={"model": "gpt-4", "response_time_ms": 1200, "confidence": 0.92},
    ),
    Output(
        value="""Here's a systematic approach to debug crashing Python code:

1. **Read the error message carefully** - Python error messages usually tell you exactly what's wrong and where.

2. **Check the stack trace** - Look at the last few lines to see which line of code caused the crash.

3. **Common crash causes**:
   - `NameError`: Variable not defined
   - `IndexError`: Accessing list element that doesn't exist
   - `KeyError`: Dictionary key not found
   - `TypeError`: Wrong data type for operation

4. **Debugging techniques**:
   ```python
   # Add print statements to trace execution
   print(f"Variable x = {x}")

   # Use try-except to catch specific errors
   try:
       risky_operation()
   except IndexError as e:
       print(f"Index error: {e}")
   ```

5. **Use a debugger** - Try `pdb.set_trace()` or your IDE's debugger to step through code line by line.

If you share the specific error message, I can give more targeted advice!""",  # noqa: E501
        metadata={"model": "gpt-4", "response_time_ms": 800, "confidence": 0.88},
    ),
]

In [8]:
# Run the complex evaluation
print("🔍 Running Detailed Quality Assessment...\n")

complex_results = evaluate(complex_test_cases, complex_outputs)

# Display results
print("📊 Evaluation Results:")
print(f"   Total test cases: {complex_results.summary.total_test_cases}")
print(f"   Completed: {complex_results.summary.completed_test_cases}")
print(f"   Errors: {complex_results.summary.error_test_cases}")
print(f"   Overall status: {complex_results.status}\n")

# Show detailed results for each test case
for i, result in enumerate(complex_results.results):
    test_case = complex_test_cases[i]
    check_result = result.check_results[0]

    # Check for errors first
    if check_result.error:
        print(f"❌ ERROR in Test Case: {test_case.id}")
        print(f"   Error Type: {check_result.error.type}")
        print(f"   Error Message: {check_result.error.message}")
        print(f"   Recoverable: {check_result.error.recoverable}")
        print("\n" + "="*80 + "\n")
        continue

    # The response fields are now directly in results
    results = check_result.results
    metadata = results.get('judge_metadata', {})

    print(f"🧪 Test Case: {test_case.id} ({test_case.metadata['domain']})")
    print(f"   Question: {test_case.input['question']}")
    print(f"   User Context: {test_case.input['user_context']}")
    print()
    print("   📊 **Quality Scores:**")
    print(f"      Overall: {results.get('overall_score', 'N/A')}/5")
    print(f"      Helpfulness: {results.get('helpfulness', 'N/A')}/5")
    print(f"      Accuracy: {results.get('accuracy', 'N/A')}/5")
    print(f"      Clarity: {results.get('clarity', 'N/A')}/5")
    print(f"      Completeness: {results.get('completeness', 'N/A')}/5")
    print()
    print("   ✅ **Strengths:**")
    for strength in results.get('strengths', []):
        print(f"      • {strength}")
    print()
    print("   ⚠️  **Areas for Improvement:**")
    for weakness in results.get('weaknesses', []):
        print(f"      • {weakness}")
    print()
    print(f"   💡 **Recommendation:** {results.get('recommendation', 'N/A')}")

    # Display metadata if available
    if metadata:
        print()
        print("   📊 **Judge Metadata:**")
        print(f"      - Cost: ${metadata.get('cost_usd', 0):.4f}")
        print(f"      - Tokens: {metadata.get('tokens_used', 0)} (in: {metadata.get('input_tokens', 0)}, out: {metadata.get('output_tokens', 0)})")  # noqa: E501
        print(f"      - Response Time: {metadata.get('response_time_ms', 0)}ms")
        print(f"      - Model: {metadata.get('model_version', 'unknown')}")

    print("\n" + "="*80 + "\n")

🔍 Running Detailed Quality Assessment...

📊 Evaluation Results:
   Total test cases: 2
   Completed: 2
   Errors: 0
   Overall status: completed

🧪 Test Case: quality_001 (AI/ML)
   Question: Can you explain how machine learning works?
   User Context: I'm a beginner with no technical background

   📊 **Quality Scores:**
      Overall: 4/5
      Helpfulness: 5/5
      Accuracy: 4/5
      Clarity: 5/5
      Completeness: 4/5

   ✅ **Strengths:**
      • The response is beginner-friendly and uses relatable analogies to explain a complex topic.
      • It clearly defines key concepts like training, data, patterns, and predictions in an accessible manner.
      • The cooking analogy is effective and makes the learning process relatable.

   ⚠️  **Areas for Improvement:**
      • While the explanation covers major points succinctly, it could involve a more structured discussion of how these concepts work together in specific instances of machine learning.
      • Additional examples or appl

## Step 5: Batch Evaluation Example

Let's demonstrate how to evaluate multiple AI responses efficiently, which is useful for:
- A/B testing different models
- Evaluating prompt engineering changes
- Quality assurance on production responses

In [9]:
# Create a batch of customer service responses to evaluate
customer_service_cases = [
    TestCase(
        id="cs_001",
        input="I'm really frustrated! My order hasn't arrived and it's been a week. What's going on?",  # noqa: E501
        metadata={"sentiment": "angry", "issue_type": "delivery_delay"},
    ),
    TestCase(
        id="cs_002",
        input="Hi, I need to return this item. It doesn't fit properly. What's your return policy?",  # noqa: E501
        metadata={"sentiment": "neutral", "issue_type": "return_request"},
    ),
    TestCase(
        id="cs_003",
        input="Your product is amazing! Just wanted to say thanks. Also, do you have any similar products?",  # noqa: E501
        metadata={"sentiment": "positive", "issue_type": "product_inquiry"},
    ),
    TestCase(
        id="cs_004",
        input="I was charged twice for the same order. This is unacceptable. I want a refund immediately.",  # noqa: E501
        metadata={"sentiment": "very_angry", "issue_type": "billing_error"},
    ),
    TestCase(
        id="cs_005",
        input="Quick question - what are your business hours? I want to call later.",
        metadata={"sentiment": "neutral", "issue_type": "information_request"},
    ),
]

# Use the same evaluation check to all test cases
customer_service_check = LLMJudgeCheck(
    prompt="""
    Evaluate this customer service response for quality and appropriateness.

    **Customer Message:** {{$.test_case.input}}
    **Customer Sentiment:** {{$.test_case.metadata.sentiment}}
    **Issue Type:** {{$.test_case.metadata.issue_type}}

    **AI Response:** {{$.output.value}}

    **Evaluation Criteria:**
    - **Helpfulness**: Does the response address the customer's needs?
    - **Empathy**: Is the tone appropriate for the customer's emotional state?
    - **Professionalism**: Is the response professional and courteous?
    - **Actionability**: Does it provide clear next steps or solutions?

    Rate each dimension and provide an overall assessment.
    """,
    response_format=DetailedQualityAssessment,
    llm_function=llm_judge_function,
)

# Simulated customer service AI responses
customer_service_outputs = [
    Output(value="I sincerely apologize for the delay with your order. I understand how frustrating this must be. Let me look into this immediately and provide you with a tracking update and expedited shipping at no cost."),  # noqa: E501
    Output(value="Of course! Our return policy allows returns within 30 days. Since the fit isn't right, you can return it for a full refund or exchange. I'll email you a prepaid return label right now."),  # noqa: E501
    Output(value="Thank you so much for the kind words! We're thrilled you love the product. Based on what you purchased, I think you'd really like our Premium Series - I'll send you some recommendations!"),  # noqa: E501
    Output(value="I apologize for this billing error - that's definitely not acceptable. I've immediately processed a refund for the duplicate charge, and you should see it in 2-3 business days. I've also added a credit to your account."),  # noqa: E501
    Output(value="Our customer service hours are Monday-Friday 8 AM to 8 PM EST, and Saturday-Sunday 10 AM to 6 PM EST. You can also reach us anytime through our 24/7 chat support!"),  # noqa: E501
]

In [ ]:
# Run batch evaluation
print("🔍 Running Customer Service Quality Batch Evaluation...\n")

batch_results = evaluate(
    test_cases=customer_service_cases,
    outputs=customer_service_outputs,
    checks=[customer_service_check],
)

# Calculate average scores across all responses
all_scores = {
    'overall': [],
    'helpfulness': [],
    'accuracy': [],
    'clarity': [],
    'completeness': [],
}

# Track total costs and tokens
total_cost = 0.0
total_tokens = 0

print("📊 **Customer Service Response Quality Report**\n")
print("| Test ID | Sentiment | Issue Type | Overall | Help | Accuracy | Clarity | Complete |")
print("|---------|-----------|------------|---------|------|----------|---------|----------|")

for i, result in enumerate(batch_results.results):
    test_case = customer_service_cases[i]

    # Check for errors first
    if result.check_results[0].error:
        print(f"| {test_case.id} | {test_case.metadata['sentiment'][:8]} | {test_case.metadata['issue_type'][:10]} | ERROR | ERROR | ERROR | ERROR | ERROR |")  # noqa: E501
        print(f"Error: {result.check_results[0].error.message}")
        continue

    # The response fields are now directly in results
    results = result.check_results[0].results
    metadata = results.get('judge_metadata', {})

    # Track metadata
    if metadata:
        total_cost += metadata.get('cost_usd', 0)
        total_tokens += metadata.get('tokens_used', 0)

    # Collect scores for averaging
    all_scores['overall'].append(results.get('overall_score', 0))
    all_scores['helpfulness'].append(results.get('helpfulness', 0))
    all_scores['accuracy'].append(results.get('accuracy', 0))
    all_scores['clarity'].append(results.get('clarity', 0))
    all_scores['completeness'].append(results.get('completeness', 0))

    # Display in table format
    print(f"| {test_case.id} | {test_case.metadata['sentiment'][:8]} | {test_case.metadata['issue_type'][:10]} | "  # noqa: E501
          f"{results.get('overall_score', 'N/A')}/5 | {results.get('helpfulness', 'N/A')}/5 | {results.get('accuracy', 'N/A')}/5 | "  # noqa: E501
          f"{results.get('clarity', 'N/A')}/5 | {results.get('completeness', 'N/A')}/5 |")

# Calculate and display averages
print("\n📈 **Average Scores Across All Responses:**")
for metric, scores in all_scores.items():
    avg_score = sum(scores) / len(scores) if scores else 0
    print(f"   {metric.title()}: {avg_score:.2f}/5")

# Find best and worst performing responses
overall_scores = all_scores['overall']
if overall_scores and any(s > 0 for s in overall_scores):
    best_idx = overall_scores.index(max(overall_scores))
    worst_idx = overall_scores.index(min(overall_scores))

    print(f"\n🏆 **Best Response:** {customer_service_cases[best_idx].id} (Score: {overall_scores[best_idx]}/5)")  # noqa: E501
    print(f"⚠️  **Needs Improvement:** {customer_service_cases[worst_idx].id} (Score: {overall_scores[worst_idx]}/5)")  # noqa: E501
else:
    print("\n⚠️ No valid scores available - all evaluations may have failed")

# Display total metadata
print(f"\n💰 **Total Evaluation Cost:** ${total_cost:.4f}")
print(f"🔢 **Total Tokens Used:** {total_tokens:,}")
if len(batch_results.results) > 0:
    print(f"📊 **Average Cost per Evaluation:** ${total_cost/len(batch_results.results):.4f}")

🔍 Running Customer Service Quality Batch Evaluation...

📊 **Customer Service Response Quality Report**

| Test ID | Sentiment | Issue Type | Overall | Help | Accuracy | Clarity | Complete |
|---------|-----------|------------|---------|------|----------|---------|----------|
| cs_001 | angry | delivery_d | 5/5 | 5/5 | 5/5 | 5/5 | 5/5 |
| cs_002 | neutral | return_req | 5/5 | 5/5 | 5/5 | 5/5 | 5/5 |
| cs_003 | positive | product_in | 5/5 | 5/5 | 5/5 | 5/5 | 5/5 |
| cs_004 | very_ang | billing_er | 4/5 | 5/5 | 5/5 | 4/5 | 4/5 |
| cs_005 | neutral | informatio | 5/5 | 5/5 | 5/5 | 5/5 | 5/5 |

📈 **Average Scores Across All Responses:**
   Overall: 4.80/5
   Helpfulness: 5.00/5
   Accuracy: 5.00/5
   Clarity: 4.80/5
   Completeness: 4.80/5

🏆 **Best Response:** cs_001 (Score: 5/5)
⚠️  **Needs Improvement:** cs_004 (Score: 4/5)

💰 **Total Evaluation Cost:** $0.0007
🔢 **Total Tokens Used:** 2,835
📊 **Average Cost per Evaluation:** $0.0001


## Step 6: Advanced Features

### Template Processing Power

Notice how we used `{{$.jsonpath}}` expressions in our prompts! This powerful feature allows dynamic prompt generation:

- `{{$.test_case.input}}` - Access the test input
- `{{$.output.value}}` - Access the AI response
- `{{$.test_case.metadata.sentiment}}` - Access nested metadata
- `{{$.output.metadata.confidence}}` - Access response metadata

### Error Handling and Robustness

The framework automatically handles:
- Invalid JSONPath expressions
- LLM API failures
- Response format validation errors
- Network timeouts and retries

### Extensibility

You can easily extend this for your use cases:
- Custom response formats for domain-specific evaluation
- Different LLM models for different types of evaluation
- Integration with your existing evaluation pipelines
- Automated report generation and alerting

## Leveraging Judge Metadata

The new interface returns both the judge's response and metadata about the LLM call. This is incredibly useful for:

1. **Cost Tracking**: Monitor and budget your LLM evaluation costs
2. **Performance Monitoring**: Track response times and optimize slow evaluations  
3. **Token Usage**: Understand token consumption for different types of evaluations
4. **Model Versioning**: Track which model versions were used for evaluations

### Example: Creating a Cost-Aware Judge Function

Here's how you can create an LLM judge function that returns comprehensive metadata:

In [11]:
# Example: Custom LLM judge function with detailed metadata
async def create_custom_judge_with_metadata(
    model_name: str = 'gpt-4o-mini',
    temperature: float = 0.0,
) -> callable:
    """Create a custom LLM judge function that returns rich metadata."""

    async def custom_judge(prompt: str, response_format: type[BaseModel]) -> tuple[BaseModel, dict[str, Any]]:  # noqa: E501
        """Custom judge with comprehensive metadata tracking."""
        import time  # noqa: PLC0415

        start_time = time.time()

        # Create client with specific configuration
        client = create_client(
            model_name=model_name,
            response_format=response_format,
            temperature=temperature,
        )

        messages = [
            system_message("You are an expert evaluator. Provide thorough assessments."),
            user_message(prompt),
        ]

        # Get evaluation from LLM
        response = await client.run_async(messages=messages)

        # Calculate detailed metadata
        end_time = time.time()
        metadata = {
            # Cost information
            "cost_usd": response.input_cost + response.output_cost,
            "input_cost_usd": response.input_cost,
            "output_cost_usd": response.output_cost,

            # Token usage
            "tokens_used": response.input_tokens + response.output_tokens,
            "input_tokens": response.input_tokens,
            "output_tokens": response.output_tokens,

            # Performance metrics
            "response_time_ms": int((end_time - start_time) * 1000),
            "llm_duration_ms": int(response.duration_seconds * 1000),

            # Model information
            "model_version": model_name,
            "temperature": temperature,

            # Additional context
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S UTC", time.gmtime()),
            "prompt_length": len(prompt),
        }

        return response.parsed, metadata

    return custom_judge

# Create a custom judge instance
custom_judge = await create_custom_judge_with_metadata(temperature=0.1)

In [12]:
# Example usage with the custom judge (fixed version)
metadata_demo_case = TestCase(
    id="metadata_demo",
    input="Explain quantum computing in simple terms",
    checks=[
        LLMJudgeCheck(
            prompt="Evaluate if this explanation is appropriate for beginners: {{$.output.value}}",
            response_format=BinaryEvaluation,
            llm_function=llm_judge_function,  # Use the working judge function instead
        ),
    ],
)

metadata_demo_output = Output(
    value="Quantum computing is like having a computer that can explore many possibilities at once, "  # noqa: E501
          "similar to how you might check multiple paths in a maze simultaneously instead of one at a time.",  # noqa: E501
)

# Run evaluation
print("🔍 Running evaluation with detailed metadata tracking...\n")
demo_result = evaluate([metadata_demo_case], [metadata_demo_output])

# Extract and display results
check_result = demo_result.results[0].check_results[0]
results = check_result.results
metadata = results.get('judge_metadata', {})

print("✅ **Judge Response:**")
print(f"   Answers Question: {results.get('answers_question')}")
print(f"   Reasoning: {results.get('reasoning')}")

print("\n📊 **Detailed Metadata:**")
for key, value in metadata.items():
    if 'cost' in key:
        print(f"   {key}: ${value:.6f}")
    elif 'time' in key or 'duration' in key:
        print(f"   {key}: {value}ms")
    else:
        print(f"   {key}: {value}")

🔍 Running evaluation with detailed metadata tracking...

✅ **Judge Response:**
   Answers Question: True
   Reasoning: The explanation effectively simplifies the complex concept of quantum computing by using a relatable analogy of a maze. It presents the idea of exploring multiple possibilities concurrently, which is a fundamental aspect of quantum computing, making it accessible for beginners.

📊 **Detailed Metadata:**
   cost_usd: $0.000057
   tokens_used: 220
   input_tokens: 166
   output_tokens: 54
   response_time_ms: 1250ms
   model_version: gpt-4o-mini


## Key Takeaways

🎯 **LLM-as-a-Judge Benefits:**
- Evaluates subjective qualities that traditional metrics can't capture
- Scales to evaluate thousands of responses consistently
- Provides detailed, actionable feedback
- Adapts to different domains and evaluation criteria

🛠️ **Implementation Tips:**
- Use structured response formats (Pydantic models) for consistency
- Provide clear evaluation criteria in your prompts
- Include relevant context and examples in templates
- Test your evaluation prompts with diverse response types

📊 **Best Practices:**
- Start with simple binary evaluations, then add complexity
- Use multiple criteria for comprehensive assessment
- Include both strengths and improvement areas in feedback
- Validate judge consistency with human evaluations

🚀 **Next Steps:**
- Integrate with your AI system's evaluation pipeline
- Experiment with different LLM models as judges
- Create domain-specific evaluation criteria
- Set up automated quality monitoring dashboards